# Incremental Dynamic Learning

## Overview

The idea behind Incremental Dynamic Learning networks is that information is not always learned at once, up front. As information is encountered, we train on the new data. Adding the new information to our existing knowlege base. Humans, are said to be able to recognize an image in 13 milliseconds after seeing it.The new image is added to their existing learned information and they do not have to retrain on all images they have ever seen.

## Example

As an example of how this can be used in the real world, I will use field biologists for my use case.

Imagine a group of field biologists are in the field and all have a mobile application which allows them to photograph wildlife they encounter, label it and forward it to a central system. The central system takes in the new photographs and trains a neural network on each piece of new data that comes in. W/out having to retrain a new model based on all past photographs. 

## Experiment 1

The experiment will consist of using a pre-trained network from TensorFlow (VGG) to generate our classification data. Our data will consist of using flower images provided with TensorFlow examples.

Once the data has be converted by the VGG network, the process of training our network begins. This will be done by feeding the network 1 image at a time, allowing the network to train for a short duration (13ms) and then move onto the next image. A fixed validation set will be used to monitor progress as the images are fed one at a time to the network.

The goal is to get an acceptable accuracy, similar to what a traditionally trained network would achieve.

## Experment 2

For the second half of this experiment will be to start with a single classification item 'UNKNOWN' and grow the classification set as needed. This will be done by checking a classificaiton index dictionary for the incoming label. If the label does not exist, then a prediction will be made with the current network. The classifcation that matches the image the best will be used to prime the initial weights of the new node to be added to the classifcation set. Once the node is added, the normal training on the image will be performed.

## Structure

The base for this experiment will be from a lesson given by Udacity on Transfer Learning. The first part of this lesson uses VGG to generate the training data for a flower classifcation system. I will use the output from that lesson to provide the training for this Expermiment.

## Get libraries and data

In [3]:
import os

import numpy as np
import tensorflow as tf

In [5]:
# read codes and labels from file
import csv

with open('labels') as f:
    reader = csv.reader(f, delimiter='\n')
    labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('codes') as f:
    codes = np.fromfile(f, dtype=np.float32)
    codes = codes.reshape((len(labels), -1))

In [7]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(labels)

labels_vecs = lb.transform(labels)

In [9]:
from sklearn.model_selection import StratifiedShuffleSplit

ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)

train_idx, val_idx = next(ss.split(codes, labels_vecs))

half_val_len = int(len(val_idx)/2)
val_idx, test_idx = val_idx[:half_val_len], val_idx[half_val_len:]

train_x, train_y = codes[train_idx], labels_vecs[train_idx]
val_x, val_y = codes[val_idx], labels_vecs[val_idx]
test_x, test_y = codes[test_idx], labels_vecs[test_idx]

In [10]:
print("Train shapes (x, y):", train_x.shape, train_y.shape)
print("Validation shapes (x, y):", val_x.shape, val_y.shape)
print("Test shapes (x, y):", test_x.shape, test_y.shape)

Train shapes (x, y): (2936, 4096) (2936, 5)
Validation shapes (x, y): (367, 4096) (367, 5)
Test shapes (x, y): (367, 4096) (367, 5)
